In [1]:
%matplotlib inline

In [2]:
import numpy as np
import nibabel as nb
import scipy.stats as stats
import matplotlib.pyplot as plt

In [3]:
def jo2allen_vol(data):
    data = np.swapaxes(np.swapaxes(data, 0,1), 1,2)
    data = np.flip(np.flip(data,1),0)
    return data

### Load gradients and cell data

In [4]:
gradient = nb.load('/home/julia/data/gradients/results/gradient0000_highres.nii.gz').get_data()
mask = nb.load('/home/julia/data/gradients/atlas/cortex/cortex_mask_25um.nii.gz').get_data()

In [5]:
template = nb.load('/home/julia/data/gradients/atlas/allen_api/template.nii.gz').get_data()

In [6]:
gradient = jo2allen_vol(gradient)
mask = jo2allen_vol(mask)
gradient[mask==0] = np.nan

In [7]:
x = np.load('/home/julia/data/gradients/atlas/epfl_cell_atlas/x2vol.npy')
y = np.load('/home/julia/data/gradients/atlas/epfl_cell_atlas/y2vol.npy')
z = np.load('/home/julia/data/gradients/atlas/epfl_cell_atlas/z2vol.npy')
types = np.load('/home/julia/data/gradients/atlas/epfl_cell_atlas/types.npy')

Types probably:
0 - excitatory neurons;
1 - inhibitory neurons;
2 - oligodendrocytes;
3 - astrocytes;
4 - microglia;

### Make a cell density map

In [8]:
cell_density = np.zeros_like(gradient)
for c in range(len(x)):
    if types[c]==0:
        voxel = (int(np.floor(x[c])), int(np.floor(y[c])), int(np.floor(z[c])))
        if not np.isnan(gradient[voxel]):
            cell_density[voxel] +=  1

KeyboardInterrupt: 

In [ ]:
nb.save(nb.Nifti1Image())

### Smooth cell density map with same kernel as gradient

In [ ]:
cell_density[mask==0]=np.nan

In [ ]:
x=250
y=100
z=150
vmin, vmax = np.nanmin(cell_density), np.nanmax(cell_density)

fig = plt.figure(figsize=(15,10))

fig.add_subplot(2,3,1)
plt.imshow(template[x,:,:],  cmap='Greys', aspect='equal');
plt.imshow(cell_density[x,:,:],  cmap="cubehelix", aspect='equal');
plt.axis('off');

fig.add_subplot(2,3,2)
plt.imshow(template[:,y,:],  cmap='Greys', aspect='equal');
plt.imshow(cell_density[:,y,:],  cmap="cubehelix", aspect='equal');
plt.axis('off');

fig.add_subplot(2,3,3)
plt.imshow(template[:,:,z].T,  cmap='Greys', aspect='equal');
plt.imshow(cell_density[:,:,z].T,  cmap="cubehelix", aspect='equal');
plt.axis('off');

fig.add_subplot(2,3,4)
plt.imshow(template[x,:,:],  cmap='Greys', aspect='equal');
plt.imshow(gradient[x,:,:],  cmap="cubehelix", aspect='equal');
plt.axis('off');

fig.add_subplot(2,3,5)
plt.imshow(template[:,y,:],  cmap='Greys', aspect='equal');
plt.imshow(gradient[:,y,:],  cmap="cubehelix", aspect='equal');
plt.axis('off');

fig.add_subplot(2,3,6)
plt.imshow(template[:,:,z].T,  cmap='Greys', aspect='equal');
plt.imshow(gradient[:,:,z].T,  cmap="cubehelix", aspect='equal');
plt.axis('off');

plt.tight_layout()

In [ ]:
stats.spearmanr(cell_density[mask==1], gradient[mask==1])